# __Step 2b: Word2Vec and BERT models__



## __Setup__

### _Imports_

In [2]:
'''
For building text classification model based on embedding of Word2Vec and BERT
'''

## For reproducibility
rand_state = 20220609

## for data
import argparse
import json
import pandas as pd
import numpy as np
import joblib
import pickle
import sys
from os import chdir
from pathlib import Path

## for bag-of-words
from sklearn import feature_extraction, feature_selection, metrics
from sklearn import model_selection
from xgboost import XGBClassifier

In [3]:
## for data
from tqdm import tqdm
from numpy.random import randint

## for word embedding with w2v
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow import keras
from tensorflow.keras import models, layers, preprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers

/home/shius/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### _Functions_

In [4]:
def read_configs(config_file):
  """Read configuration file and return a config_dict"""
  # required
  config_dict = {'lang_model':0,
                 'proj_dir':0,
                 'work_dir':0,
                 'corpus_combo_file':0,
                 'rand_state':0,
                 'p_threshold':0,
                 'xg_param':0,
                 'n_splits':0,
                 'xg_param_comb':0,
                 'n_jobs':0,}

  # Read config file and fill in the dictionary
  with open(config_file, 'r') as f:
    configs     = f.readlines()
    for config in configs:
      if config.strip() == "" or config[0] == "#":
        pass
      else:
        config = config.strip().split("=")
        if config[0] in config_dict:
          config_dict[config[0]] = eval(config[1])

  # Check if any config missing
  missing = 0
  for config in config_dict:
    if config_dict[config] == 0:
      print("  missing:", config)
      missing += 1
    else:
      print("  ", config, "=", config_dict[config])

  if missing == 0:
    print("  all config available")
  else:
    print("  missing config, QUIT!")
    sys.exit(0)

  return config_dict


def split_train_test(corpus_combo_file, rand_state):
  '''Load data and split train test
  Args:
    corpus_combo_file (str): path to the json data file
    rand_state (int): for reproducibility
  Return:
    train_ori, test_ori, train_cln, test_cln (pandas dataframes): for the
      original and clean texts, training and testing splits.
  '''
  # Load json file
  with corpus_combo_file.open("r+") as f:
      corpus_combo_json = json.load(f)

  # Convert json back to dataframe
  corpus_combo = pd.read_json(corpus_combo_json)

  corpus_ori = corpus_combo[['label','txt']]
  train_ori, test_ori = model_selection.train_test_split(corpus_ori, 
      test_size=0.2, stratify=corpus_ori['label'], random_state=rand_state)

  # Cleaned corpus
  corpus_cln = corpus_combo[['label','txt_clean']]
  corpus_cln.rename(columns={'txt_clean': 'txt'}) # make col names consistent
  train_cln, test_cln = model_selection.train_test_split(corpus_cln, 
      test_size=0.2, stratify=corpus_cln['label'], random_state=rand_state)

  return train_ori, test_ori, train_cln, test_cln


def select_feat(X_train, y_train, vectorizer, p_threshold):
  '''Select features based on chi-square test results
  Args:
    X_train (pandas series): the txt column in the training data frame
    y_train (pandas series): the label column in the training data frame
    vecorizer: fitted with original X_train and returned from get_vectorizer()
    p_threshold (float): p is derived from chi-square test. Features with p <= 
      p_threshold_s are selected.
  Return:
    X_names (list): names of selected features
  '''
  y            = y_train
  X_names      = vectorizer.get_feature_names_out()
  dtf_features = pd.DataFrame()
  for cat in np.unique(y):
    _, p = feature_selection.chi2(X_train, y==cat)
    dtf_features = pd.concat([dtf_features, 
                pd.DataFrame({"feature":X_names, "p":p, "y":cat})])
    dtf_features = dtf_features.sort_values(
                ["y","p"], ascending=[True,False])
    dtf_features = dtf_features[dtf_features["p"] <= p_threshold]
  
  X_names = dtf_features["feature"].unique().tolist()

  return X_names


def run_xgboost(X_train, y_train, config_dict):
  '''Do hyperparameter tuning and cross-validation of XgBoost models
  Args:
    X_train (pandas dataframe): features
    y_train (pandas series): labels
    config_dict (dict): from read_config()
  Return:
    rand_search (RandomizedSearchCV): fitted obj
  '''

  rand_state = config_dict["rand_state"]
  param_grid = config_dict["xg_param"] 
  n_splits   = config_dict["n_splits"]
  param_comb = config_dict["xg_param_comb"]
  n_jobs     = config_dict["n_jobs"]

  # Initialize classifier
  # 06/11/2022: the silent parameter is deprecated, use verbosity=0
  xgb = XGBClassifier(learning_rate=0.02, 
                      n_estimators=600, 
                      objective='binary:logistic',
                      verbosity=1, 
                      nthread=1)
  # Initilize stratified k fold obj
  skf = model_selection.StratifiedKFold(n_splits=n_splits, 
                      shuffle = True, random_state = rand_state)
  # initiate randomized search CV obj
  rand_search = model_selection.RandomizedSearchCV(
                      xgb                , param_distributions = param_grid, 
                      n_iter = param_comb, scoring      = 'f1', 
                      n_jobs = n_jobs    , cv = skf.split(X_train,y_train), 
                      verbose = 3        , random_state =rand_state)
  # Train
  rand_search.fit(X_train, y_train)

  return rand_search


def run_main_function(work_dir, train, test, txt_flag, config_dict):

  # Get the training/testing corpus and labels
  if txt_flag == "ori":
    X_train = train['txt']
    X_test  = test['txt']
  else:
    X_train = train['txt_clean']
    X_test  = test['txt_clean']

  y_train = train['label']
  y_test  = test['label']

  # get vectorizer parameter list
  p_threshold = config_dict['p_threshold']
  param_list  = get_hyperparameters(config_dict['vec_param'], p_threshold)
  lang_model  = config_dict['lang_model']

  # iterate through different parameters
  with open(work_dir / f"scores_{txt_flag}", "w") as f:
    f.write("run\ttxt_flag\tlang_model\tparameters\tnum_feat\tcv_f1\t" +\
            "test_f1\tmodel_name\n")
    run_num = 0
    for param in param_list:
      print(f"\n## param: {param}")
      best_score, num_select, model_name, test_score = run_pipeline(
        work_dir, X_train, y_train, X_test, y_test, param, txt_flag, config_dict)

      f.write(f"{run_num}\t{txt_flag}\t{lang_model}\t{str(param)}\t"+\
              f"{num_select}\t{best_score}\t{test_score}\t{model_name}\n")

      run_num += 1


def run_pipeline(work_dir, X_train, y_train, X_test, y_test, param, txt_flag,
                 config_dict):
  '''Carry out the major steps'''

  # Get vectorizer and fitted X_train
  print("  extract features by fitting a vectorizer")
  lang_model = config_dict['lang_model']
  vectorizer, X_train_vec = extract_feat(X_train, param, lang_model)
  print("    train dim:", X_train_vec.shape)

  # Get selected feature names
  print("  select features")
  p_threshold = config_dict['p_threshold']
  X_names     = select_feat(X_train_vec, y_train, vectorizer, p_threshold)
  num_select  = len(X_names)
  print('    total selected:', num_select)

  # Refit vectorizer with selected features and re-transform X_train
  print("  refit vectorizer with training data and transform")
  vectorizer_sel, X_train_vec_sel = extract_feat(X_train, vocab=X_names)
  print("    train dim:", X_train_vec_sel.shape)

  # Also apply the refitted vecorizer to testing data
  print("  transform testing data")
  X_test_vec_sel = vectorizer_sel.transform(X_test)
  print("    test dim:", X_test_vec_sel.shape)

  # Get xgboost model and cv results
  print("  cross-validation and tuning with xgboost")
  rand_search = run_xgboost(X_train_vec_sel, y_train, config_dict)

  best_est   = rand_search.best_estimator_
  best_param = rand_search.best_params_
  best_score = rand_search.best_score_
  print("    best F1:", best_score)
  print("    best param:", best_param)

  # Save the best model
  print ("  save model")
  param_str  = \
      f"{int(param[0])}-{'to'.join(map(str,param[1]))}-{param[2]}"

  model_name = work_dir / f'model_{txt_flag}_{param_str}.sav'
  joblib.dump(best_est, model_name)

  # Get testing results: This is not for tuning/selection purpose but because
  # X_test is transformed by vectorizer for each parameter combination. If
  # the testing set is not evaluated now, things just get too complicated.
  # Keep in mind that the testing F1s will not be compared against each other.
  print("  get testing f1")
  y_pred = best_est.predict(X_test_vec_sel)
  test_score = metrics.f1_score(y_test, y_pred)

  # provide some space between runs
  print('\n')

  return best_score, num_select, model_name, test_score

### _Get training/testing split_

In [5]:
config_file = Path("config_w2v_bert.txt")

print("\nRead configuration file...")
config_dict = read_configs(config_file)

# Set up working directory and corpus file location
proj_dir          = Path(config_dict['proj_dir'])
work_dir          = proj_dir / config_dict['work_dir']
corpus_combo_file = work_dir / config_dict['corpus_combo_file']

# For reproducibility
rand_state = config_dict['rand_state']

# Split train/test for original and cleaned text
print("\nRead file and split train/test...")
train_ori, test_ori, train_cln, test_cln = split_train_test(
                                              corpus_combo_file, rand_state)


Read configuration file...
   lang_model = w2v
   proj_dir = /home/shius/projects/plant_sci_hist
   work_dir = 2_text_classify
   corpus_combo_file = corpus_combo
   rand_state = 20220609
   p_threshold = 0.0001
   xg_param = {'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]}
   n_splits = 5
   xg_param_comb = 5
   n_jobs = 12
  all config available

Read file and split train/test...


## __Word2Vec-based model__

### _Testing with cleaned text_

Starting out, I was testing with the original text. When looking into most similar words to "jasmonate" there are substantial issue. Top 20 most similar:

````Python
[('(MeJA)', 0.7294234037399292), ('jasmonic', 0.6400792002677917), ('methyl', 0.638791024684906), ('JA', 0.6380835175514221), ('(JA)', 0.6271078586578369), ('(MeJA),', 0.6264932155609131), ('jasmonate,', 0.6005000472068787), ('(MeJA).', 0.5945847034454346), ('ZIM-domain', 0.5596743226051331), ('jasmonate-', 0.5553220510482788), ('salicylic', 0.5499346256256104), ('JA-isoleucine', 0.5474823713302612), ('12-oxophytodienoic', 0.5435537099838257), ('aminocyclopropane', 0.542698860168457), ('methyl-jasmonate', 0.5406786799430847), ('methyljasmonate', 0.5398091077804565), ('wounding', 0.5363242030143738), ('jasmonate..', 0.5355076789855957), ('acid-isoleucine', 0.531741738319397), ('(MeSA)', 0.5283530950546265)]
```

So for the next test run repeat and the real run, only cleaned text is used.

#### Get uni-, bi-, and tri-grams

In [7]:
# Fit my own Word2Vec model with my corpus
corpus = train_cln['txt_clean'] # pandas Series
type(corpus), corpus[:2]

(pandas.core.series.Series,
 853778     update exertional hyponatremia active componen...
 1165206    stable megadalton toctic supercomplexes major ...
 Name: txt_clean, dtype: object)

In [9]:
## create list of lists of unigrams
lst_corpus = []
for string in corpus:

   # Q: lst_words and lst_grams are the same, what's the point?
   lst_words = string.split()
   
   #lst_grams = [" ".join(lst_words[i:i+1]) 
   #            for i in range(0, len(lst_words), 1)]
   #lst_corpus.append(lst_grams)

   lst_corpus.append(lst_words)
print(lst_corpus[0][:20])

['update', 'exertional', 'hyponatremia', 'active', 'component', 'u', 'armed', 'force', '19992012', '1999', '2012', '1333', 'incident', 'diagnosis', 'exertional', 'hyponatremia', 'among', 'active', 'component', 'member']


In [12]:
## detect bigrams and trigrams

bigrams_detector  = gensim.models.phrases.Phrases(lst_corpus, 
                                                  delimiter=" ", 
                                                  min_count=5, 
                                                  threshold=10)
bigrams_detector  = gensim.models.phrases.Phraser(bigrams_detector)

In [13]:
# Note that the input the trigrams_detector is output of the bigrams_detector
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
                                                  delimiter=" ", 
                                                  min_count=5, 
                                                  threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

#### _Initialize Word2Vec model_

- `sequences`: lst_corpus
- `vector_size`: dimension of word embeddings
- `window`: max distance between the current and the predicted words in a sentence
- `min_count`: ignore all words with total frquency lower than this.
  - [Discussion on seeting min_count](https://stackoverflow.com/questions/50723303/how-is-word2vec-min-count-applied)
- `sg`: history algorithm, 1: skip-gram, otherwise CBOW.

Following [this](https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook).

In [19]:
model_w2v = gensim.models.Word2Vec(vector_size=300, window=8, 
                                   min_count=20, sg=1, epochs=30, workers=16,
                                   seed=rand_state)

In [20]:
# Building the Vocabulary Table
model_w2v.build_vocab(lst_corpus, progress_per=10000)

In [23]:
# Train w2v model
model_w2v.train(lst_corpus, total_examples=model_w2v.corpus_count, epochs=30,
                report_delay=1)

In [ ]:
# Save the w2v model
#with open(work_dir / "model_ori_w2v", "wb") as f:
#    pickle.dump(model_w2v, f)

# Genism build in functionfor pickling
model_w2v.save(work_dir / "model_ori_w2v")

In [ ]:
# Testing the w2v model
# Here there is problem with stop words. Like 'jasmonate..', '(MeJA)', and other
# variants. So should use the cleaned text.
example = "jasmonate"
print(len(model_w2v.wv[example]))
print(model_w2v.wv.most_similar(example, topn=20))

#### _Train tokenizer_

In [ ]:
# intialize tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(
                        lower=True, 
                        split=' ', 
                        oov_token="NaN", 
                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

# tokenize corpus 
tokenizer.fit_on_texts(lst_corpus)

# get token dictionary, with token as key, index number as value
dic_vocab_token = tokenizer.word_index

#### _Turn texts into index numbers_

Transforms each text in texts to a sequence of integers.

In [ ]:
lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)
print(lst_corpus[0][:10])
print(lst_text2seq[0][:10])

In [ ]:
# The index numbers are the values from the token dictionary
# Note that these are lowercased
dic_vocab_token['update:'], dic_vocab_token['exceptional']

#### _Check min, max, avg len_

In [ ]:
### NOT Run ###

minlen = 100; maxlen = 0; totlen = 0
lst_0  = []   # index of sequences with zero lengths

for idx in tqdm(range(len(lst_text2seq))):
    slen   = len(lst_text2seq[idx])
    totlen +=slen
    if slen > maxlen: maxlen = slen
    if slen < minlen: 
        if slen == 0: lst_0.append(idx)
        else: minlen = slen
print(f'Min:{minlen}, Max:{maxlen}, Avg:{totlen/len(lst_text2seq)}')
print('Zero length:', lst_0)

#### _Pad or trucate sequences_

In [ ]:
X_train_w2v = keras.preprocessing.sequence.pad_sequences(
                    lst_text2seq,      # List of sequences, each a list of ints 
                    maxlen=500,         # maximum length of all sequences
                    padding="post",    # 'pre' or 'post' 
                    truncating="post") # remove values from sequences > maxlen
X_train_w2v.shape

#### _Create embedding matrix_

In [ ]:
## start the matrix (length of vocabulary x vector size) with all 0s

embeddings = np.zeros((len(dic_vocab_token)+1, 300))
not_in_emb = {}
for word, idx in dic_vocab_token.items():
    ## update the row with vector
    try:
        embeddings[idx] =  model_w2v.wv[word]
    ## if word not in model then skip and the row stays all 0s
    except KeyError:
        not_in_emb[word] = 1

not_in_emb # Q: How did this got into the corpus??

#### _Set up ANN_

The model contains:
- An embedding layer:
  - Sequences as input (15 tokens, including padding)
  - Word (embedding?) vectors as weights (what??)
  - Embedding as output (15x300).
- An attention layer
  - Capture the eughts of each instance for building an explaniner.
  - Not needed for the predictions.
- Two layers of bidirectional LSTM.
- Two final dense layer to predict probabilities of classes

In [ ]:
def get_w2v_emb_model(embeddings):

    ## code attention layer
    def attention_layer(inputs, neurons):
        x = layers.Permute((2,1))(inputs)
        x = layers.Dense(neurons, activation="softmax")(x)
        x = layers.Permute((2,1), name="attention")(x)
        x = layers.multiply([inputs, x])
        return x

    ## input
    x_in = layers.Input(shape=(15,)) ## embedding
    x = layers.Embedding(input_dim=embeddings.shape[0],  
                        output_dim=embeddings.shape[1], 
                        weights=[embeddings],
                        input_length=15, trainable=False)(x_in)

    ## apply attention
    x = attention_layer(x, neurons=15)

    ## 2 layers of bidirectional lstm
    x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                            return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)

    ## final dense layers
    x = layers.Dense(64, activation='relu')(x)
    y_out = layers.Dense(3, activation='softmax')(x)

    ## Initialize and compile model
    model = models.Model(x_in, y_out)
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

    return model


In [ ]:
model_emb = get_w2v_emb_model(embeddings)
model_emb.summary()

#### _Convert text labels to numeric ones_

In [ ]:
## encode y
#  This is the class label, not sure why inverse is done.
dic_y_mapping = {n:label for n,label in enumerate(np.unique(y_train))}
inverse_dic   = {v:k for k,v in dic_y_mapping.items()}
inverse_dic

In [ ]:
# Convert text labels to numeric ones.
y_train_label = np.array([inverse_dic[y] for y in y_train])
X_train_w2v.shape, len(y_train_label)

#### _Train model_

In [ ]:
## train
history = model_emb.fit(x=X_train_w2v, y=y_train_label, batch_size=256, 
                        epochs=10, shuffle=True, verbose=1, 
                        validation_split=0.3)

#### _Plot loss and accuracy_


In [ ]:
his_keys = history.history.keys()
his_keys

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
df_history_loss = pd.DataFrame(history.history)[['loss','val_loss']]
df_history_loss.plot(ax=ax1)
df_history_accu = pd.DataFrame(history.history)[['accuracy','val_accuracy']]
df_history_accu.plot(ax=ax2)
ax1.grid(True); ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss')
ax2.grid(True); ax2.set_xlabel('Epoch'); ax2.set_ylabel('Cross entropy')
plt.show()

#### _Evaluate model_

In [ ]:
y_pred_prob_w2v = model_emb.predict(X_test_w2v)

# q: Why use dic_y_mapping instead of inverse_dic???
y_pred_w2v      = [dic_y_mapping[np.argmax(pred)] for pred in y_pred_prob_w2v]

eval(y_test, y_pred_w2v, y_pred_prob_w2v, plot_auc=1)